# Mini Wikipedia Vector Retrieval System: [Pizza Napoletana]
Sistema che scarica e pulisce la pagina di Wikipedia sulla Pizza Napoletana, le indicizza con FAISS e identifica i frammenti più rilevanti calcolando la distanza L2. 

### Configurazione dati

In [1]:
from langchain_community.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings   
from langchain_community.vectorstores import FAISS
from langchain.docstore.document import Document
import bs4
import re
import shutil
import os

#RESET INDICE
if os.path.exists("faiss_index_pizza"):
    shutil.rmtree("faiss_index_pizza")

#CARICAMENTO PAGINA
url = "https://it.wikipedia.org/wiki/Pizza_napoletana"
print(f"🍕🍕🍕 Sto attingendo alla mie conoscenze da pizzaiolo napoletano...🍕🍕🍕")


loader = WebBaseLoader(
    url,
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(id="bodyContent") #prende il corpo e non il resto che darebbe problemi
    )
)
data = loader.load()

#PULIZIA MANUALE
testo_sporco = data[0].page_content

testo_pulito = re.sub(r'\[.*?\]', '', testo_sporco)
testo_pulito = re.sub(r'\n\s*\n', '\n\n', testo_pulito)

print(f"Testo scaricato. Lunghezza: {len(testo_pulito)} caratteri.")


data_pulita = [Document(page_content=testo_pulito, metadata=data[0].metadata)]

#SPLITTING
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=600,      
    chunk_overlap=150,  
    separators=["\n\n", "\n", ". ", "; ", " "] 
)

chunks = text_splitter.split_documents(data_pulita)
print(f"Ho creato {len(chunks)} chunks.")

#SALVATAGGIO
emb = HuggingFaceEmbeddings(model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")
vectorstore = FAISS.from_documents(chunks, emb)
vectorstore.save_local("faiss_index_pizza")
print(" Sono pronto.")

/Users/andreademarco/formazione_sou/track_pi_greco/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
USER_AGENT environment variable not set, consider setting it to identify your requests.
/Users/andreademarco/formazione_sou/track_pi_greco/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


🍕🍕🍕 Sto attingendo alla mie conoscenze da pizzaiolo napoletano...🍕🍕🍕
Testo scaricato. Lunghezza: 9516 caratteri.
Ho creato 26 chunks.
 Sono pronto.


## Chat 
### Domande suggerite: "Qual è la peculiarità della pizza napoletana? Con cosa è condita la pizza alla marinara? Dove deve avvenire la cottura della pizza?"

In [2]:
#Ricarico indice per sicurezza
emb = HuggingFaceEmbeddings(model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")
vectorstore = FAISS.load_local("faiss_index_pizza", emb, allow_dangerous_deserialization=True)

print(f"\n🍕🍕🍕 Bot napoletano pronto! Fammi una domanda. Se vuoi uscire, digita 'esci' o 'stop' o 'exit'. Vai:")
print("=" * 60)

while True:
    user_input = input("Domanda: ")
    if user_input.lower() in ["esci", "stop", "exit"]: break
    if not user_input.strip(): continue

    #stampo i migliori k
    results = vectorstore.similarity_search_with_score(user_input, k=4)
    
    print(f"\n🍕🍕🍕 Risultati per: '{user_input}'")
    
    found_answer = False
    for i, (doc, score) in enumerate(results):
        if score > 15: continue      #filtro quelli senza senso
            
        print(f"--- #{i+1} (Dist: {score:.2f}) ---")
        print(f"\"{doc.page_content}\"")
        found_answer = True
        print()
    
    if not found_answer:
        print("Nessun risultato rilevante trovato.")
    
    print("-" * 60)


🍕🍕🍕 Bot napoletano pronto! Fammi una domanda. Se vuoi uscire, digita 'esci' o 'stop' o 'exit'. Vai:

🍕🍕🍕 Risultati per: 'qual è la peculiarità della pizza napoletana? '
--- #1 (Dist: 8.16) ---
"Pizza Margherita: la più famosa e iconica delle pizze napoletane, è condita con pomodoro, mozzarella fiordilatte, basilico fresco e un filo d’olio extravergine d’oliva; è stata creata in onore della Regina Margherita di Savoia nel 1889;
Pizza alla marinara: una delle più antiche, è condita con pomodoro, aglio, origano e olio extravergine d’oliva; è chiamata così perché era il pasto tipico dei marinai.
Cottura"

--- #2 (Dist: 8.98) ---
"(Art. 6 della disciplinare per la definizione di standard internazionali per l'ottenimento del marchio "Pizza Napoletana STG")
La lavorazione e gli ingredienti della verace pizza napoletana artigianale sono definiti nella norma UNI 10791:98 e sono stati predisposti dall'Associazione Verace Pizza Napoletana che dal 1984 promuove la conoscenza della verace pizza na